In [131]:
#Certain collection ID of interest copied from the URL of the collection in the web browser
collection_id="0f528c8a-a25c-4840-8fa3-d156fa11086f"

In [132]:
import requests

domain_name = "cellxgene.cziscience.com"
site_url = f"https://{domain_name}"
api_url_base = f"https://api.{domain_name}"
dataset_path = f"/curation/v1/collections/{collection_id}"
dataset_url = f"{api_url_base}{dataset_path}"
headers = {"Content-Type": "application/json"}
res = requests.get(url=dataset_url, headers=headers)
res.raise_for_status()
res_content = res.json()

In [135]:
dataset_ids = []
for dataset in res_content["datasets"]:
    dataset_ids.append(dataset["dataset_version_id"])

In [136]:
#first dataset for illustration
dataset_id = dataset_ids[0]

['51101a5a-7f7e-4559-b67a-5f5fd484569c',
 'b2f1886e-a3aa-4d48-80c9-dcdd44e89e02']

In [137]:
import cellxgene_census
census = cellxgene_census.open_soma()

The "stable" release is currently 2023-12-15. Specify 'census_version="2023-12-15"' in future calls to open_soma() to ensure data consistency.


In [139]:
import cellxgene_census.experimental.ml as census_ml
import tiledbsoma as soma
import numpy as np

experiment = census["census_data"]['homo_sapiens']

random_idx = np.random.randint(0,50000,1000)
experiment_datapipe = census_ml.ExperimentDataPipe(
    experiment,
    measurement_name="RNA",
    obs_query=soma.AxisQuery(value_filter=f"dataset_id == '{dataset_id}'"), #value_filter="tissue_general == 'tongue' and is_primary_data == True",
    #obs_column_names=["cell_type"],
    batch_size=128,
    shuffle=True,
    soma_chunk_size=10_000,
)

In [140]:
experiment_datapipe.shape

(0, 60664)

In [143]:
#also doesnt work with gget
import gget
adata = gget.cellxgene(
    dataset_id = f'{dataset_id}'
)

09:21:28 - INFO - Fetching AnnData object from CZ CELLxGENE Discover. This might take a few minutes...


The "stable" release is currently 2023-12-15. Specify 'census_version="2023-12-15"' in future calls to open_soma() to ensure data consistency.


In [144]:
adata

AnnData object with n_obs × n_vars = 0 × 60664
    obs: 'dataset_id', 'assay', 'suspension_type', 'sex', 'tissue_general', 'tissue', 'cell_type', 'is_primary_data'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length', 'nnz', 'n_measured_obs'

In [ ]:
#Generally the approach works if a dataset_id from this list is used
sex_cell_metadata = census["census_data"]["homo_sapiens"].obs.read(column_names=["dataset_id"]).concat().to_pandas()
sex_cell_metadata.drop_duplicates()